# Installing Necessary Libraries

In [1]:
! pip install sentence_transformers pypdf faiss-cpu
! pip install langchain langchain-openai
! pip install langchain_community
! pip install -U langchain-huggingface

# Importing necessary libraries

In [2]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
import dotenv

# Open AI Environment

In [3]:
dotenv.load_dotenv()

True

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

/Users/taimourabdulkarim/miniconda3/envs/mlp/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Importing Dataset

In [5]:
loader = PyPDFLoader("Dataset/RAG Input Doc.pdf")
documents = loader.load()
documents

[Document(page_content="Title:  MeshAnything: Artist -Created Mesh Generation with Autoregressive Transformers  \nAuthors:  buaacyw/meshanything  \nDate:  14 Jun 2024  \nDescription:  Recently, 3D assets created via reconstruction and generation have matched the \nquality of manually crafted assets, highlighting their potential for replacement.  \nStats:  417, 5.09 stars / hour  \nCategories:  Decoder  \nLinks:  Paper, Code  \n \nTitle:  Accessing GPT -4 level Mathematical Olympiad Solutions via Monte Carlo Tree Self -\nrefine with LLaMa -3 8B  \nAuthors:  trotsky1997/mathblackbox  \nDate:  11 Jun 2024  \nDescription:  This paper introduces the MCT Self -Refine algorithm, an innovative integration of \nLarge Language Models (LLMs) with Monte Carlo Tree Search (MCTS), designed to enhance \nperformance in complex mathematical reasoning tasks.  \nStats:  279, 2.35 stars / hour  \nCategories:  Decision Making, GSM8K +2  \nLinks:  Paper, Code  \n \nTitle:  TextGrad: Automatic 'Differentiati

In [6]:
# Split text
text = RecursiveCharacterTextSplitter().split_documents(documents)
text

[Document(page_content="Title:  MeshAnything: Artist -Created Mesh Generation with Autoregressive Transformers  \nAuthors:  buaacyw/meshanything  \nDate:  14 Jun 2024  \nDescription:  Recently, 3D assets created via reconstruction and generation have matched the \nquality of manually crafted assets, highlighting their potential for replacement.  \nStats:  417, 5.09 stars / hour  \nCategories:  Decoder  \nLinks:  Paper, Code  \n \nTitle:  Accessing GPT -4 level Mathematical Olympiad Solutions via Monte Carlo Tree Self -\nrefine with LLaMa -3 8B  \nAuthors:  trotsky1997/mathblackbox  \nDate:  11 Jun 2024  \nDescription:  This paper introduces the MCT Self -Refine algorithm, an innovative integration of \nLarge Language Models (LLMs) with Monte Carlo Tree Search (MCTS), designed to enhance \nperformance in complex mathematical reasoning tasks.  \nStats:  279, 2.35 stars / hour  \nCategories:  Decision Making, GSM8K +2  \nLinks:  Paper, Code  \n \nTitle:  TextGrad: Automatic 'Differentiati

In [7]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5", encode_kwargs={"normalize_embeddings": True}
)

/Users/taimourabdulkarim/miniconda3/envs/mlp/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/taimourabdulkarim/miniconda3/envs/mlp/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# Create a vectorstore
vectorstore = FAISS.from_documents(text, embeddings)

# Save the documents and embeddings
vectorstore.save_local("vectorstore.db")

# create retriever
retriever = vectorstore.as_retriever()

In [9]:
# create document chain
template = """"
You are an assistant for question-answering tasks.
Use the provided context only to answer the following question:

<context>
{context}
</context>

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)
doc_chain = create_stuff_documents_chain(llm, prompt)

In [10]:
chain = create_retrieval_chain(retriever, doc_chain)

In [11]:
questions = [
    "What is the focus of the 'MeshAnything' project?",
    "Which paper discusses the integration of Large Language Models with Monte Carlo Tree Search?",
    "What advancements does the 'VideoLLaMA 2' paper propose?",
    "Which paper was published most recently?",
    "Identify a paper that deals with language modeling and its scalability.",
    "Which paper aims at improving accuracy in Google-Proof Question Answering?",
    "List the categories covered by the paper titled 'TextGrad: Automatic \"Differentiation\" via Text'.",
    "Which paper received the highest number of stars per hour?",
    "Who is Emma Stone?",  # Added question
]

for question in questions:
    response = chain.invoke({"input": question})
    if response["answer"] and "Emma Stone" not in response["answer"]:
        print(f"Question: {question}\nAnswer: {response['answer']}\n\n")
    else:
        print(f"Question: {question}\nNo information\n\n")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Question: What is the focus of the 'MeshAnything' project?
Answer: The focus of the 'MeshAnything' project is on artist-created mesh generation with autoregressive transformers.


Question: Which paper discusses the integration of Large Language Models with Monte Carlo Tree Search?
Answer: The paper titled "Accessing GPT-4 level Mathematical Olympiad Solutions via Monte Carlo Tree Self-refine with LLaMa-38B" by trotsky1997/mathblackbox discusses the integration of Large Language Models with Monte Carlo Tree Search.


Question: What advancements does the 'VideoLLaMA 2' paper propose?
Answer: The 'VideoLLaMA 2' paper proposes advancements in spatial-temporal modeling and audio understanding in video and audio-oriented tasks.


Question: Which paper was published most recently?
Answer: The paper titled "MeshAnything: Artist-Created Mesh Generation with Autoregressive Transformers" was published most recently on 14 Jun 2024.


Question: Identify a paper that deals with language modeling an